# Reduced order model for a lithium ion cell with uniform reaction rate approximation

## Introduction
This model considers a Li-ion cell connected to an external load in a circuit. We consider that that a 3-dimensional cylindrical cell can be opened up as a sheet of paper of negligible thickness, now also assume that no change in any physical quantity occurs along the width. This turn our 3D system to a 1D one. 

Now according to standard conventions let's assume that our cell consists of **3 regions**, the **-ve electrode**, **the separator** and the **+ve electrode**. Also assume that the cell has collectors at end of either electrode. We'll be having the standard reactions in the +ve and -ve electrodes as below:

*-ve electrode:*

<div align=center>LiC₆ → x.Li⁺ + Li₁₋ₓC₆ + x.e⁻</div>

*+ve electrode:*

<div align=center>x.Li⁺ + LiMO₂ + x.e⁻ → Li₁₊ₓMO₂</div>

With this our system should look like below.

<div align=center><img src="../assets/senthil/li_cell.png"></div>

Assume that we have `x = 0` at the -ve electrode and `x = L` at the +ve electrode, L being the cell length. Length of the regions from left to right being lₙ, lₛ and lₚ respectively.

## Importing necessary libraries
To get started import the CaseStudy's senthil model (as sm for convinence) as that's the model which we're simulating. Also import Gadfly as that's the plotting library we're gonna use. To see the model constants [check here.](../src/senthil/constants/constants.jl)

In [1]:
import LiionBatteryModels.SenthilModel as sm
using Gadfly

set_default_plot_size(800px, 600px)

## Cell Voltage for different cases
In this example we'll be showing the cases of *constant current* and *discrete time-series current*. The first describes all the internal model variables also while the latter just shows you how to calculate the voltage through the necessary steps.

Essentially you can need to get values of 4 variables q₂ᵢₖ, c₂ᵢₖ, c₁ₖ and c₁ₖᵣ to calculate any cell variable (including the cell voltage) for a given current and time interval.

### Constant current case
In this case we'll be taking a current of 1C or 13.5mA over a time range of 3400s with intervals of 1s.

In [2]:
I = 13.5
t = Float64.(0:1:3400)
tspan = 1

1

#### Calculation of Interfacial Flux (q₂ᵢₖ)
The interfacial concentrations q₂ᵢₙ and q₂ᵢₚ (in -ve and +ve electodes respectively) are calculated by solving the ODE obtained after profile approximations with initial values of 0, 0. The results are also plotted.

In [3]:

q₂ᵢₖ = sm.InterfacialFlux(t, tspan, I)


plot(
    layer(x=t, y=q₂ᵢₖ.q₂ᵢₙ, Geom.line, color=[colorant"orange"]),
    layer(x=t, y=q₂ᵢₖ.q₂ᵢₚ, Geom.line, color=[colorant"lightblue"]),
    Guide.title("Variation of Interfacial Flux in Electrodes"),
    Guide.xlabel("Time (in s)"),
    Guide.ylabel("q₂ᵢₖ"),
    # Guide.manual_color_key(
    #     "Legend", 
    #     [
    #         "Interfacial flux in -ve electode: q₂ᵢₙ", 
    #         "Interfacial flux in +ve electode: q₂ᵢₚ"
    #     ], 
    #     ["orange", "lightblue"]
    # )
)

#### Calculation of Interfacial Concentration (c₂ᵢₖ)
Having calculated q₂ᵢₖ, we can calulate c₂ᵢₙ and c₂ᵢₚ via a set of linear algeabric equations. The results are plotted.

In [ ]:
c₂ᵢₖ = sm.InterfacialConc(q₂ᵢₖ)


plot(
    layer(x=t, y=c₂ᵢₖ.c₂ᵢₙ, Geom.line, color=[colorant"orange"]),
    layer(x=t, y=c₂ᵢₖ.c₂ᵢₚ, Geom.line, color=[colorant"lightblue"]),
    Guide.title("Variation of interfacial concentration in electrodes"),
    Guide.xlabel("Time (in s)"),
    Guide.ylabel("c₂ᵢₖ"),
    # Guide.manual_color_key(
    #     "Legend", 
    #     [
    #         "Interfacial concentration in -ve electode: q₂ᵢₙ", 
    #         "Interfacial concentration in +ve electode: q₂ᵢₚ"
    #     ], 
    #     ["orange", "blue"]
    # )
)

#### Calculation of various electrolyte concentration profiles
Having obtained q₂ᵢₖ and c₂ᵢₖ, we can plot the internal parameters like c₂₀ₜ, c₂ₘₜ and c₂ₗₜ, which are electrolyte concentrations at x = 0, lₙ + lₛ/2 and L respectively.

In [ ]:
c₂₀ₜ = sm.c₂₀ₜ(c₂ᵢₖ, q₂ᵢₖ)
c₂ₘₜ = sm.c₂ₘₜ(c₂ᵢₖ, q₂ᵢₖ)
c₂ₗₜ = sm.c₂ₗₜ(c₂ᵢₖ, q₂ᵢₖ)

plot(
    layer(x=t, y=c₂₀ₜ, Geom.line, color=[colorant"orange"]),
    layer(x=t, y=c₂ₘₜ, Geom.line,color=[colorant"green"]),
    layer(x=t, y=c₂ₗₜ, Geom.line,color=[colorant"lightblue"]),
    Guide.title("Variation of electrolyte concentrations in electrodes"),
    Guide.xlabel("Time (in s)"),
    Guide.ylabel("Concentration"),
    # Guide.manual_color_key(
    #     "Legend", 
    #     [
    #         "Solid concentration in -ve electode: q₂ᵢₙ", 
    #         "Solid concentration in +ve electode: q₂ᵢₚ"
    #     ], 
    #     ["orange", "lightblue"],
    # )
)

#### Calculation of Solid Concentration (c₁ₖ)

In [ ]:
c₁ₖ = sm.SolidConcentration(t, tspan, I)

plot(
    layer(x=t, y=c₁ₖ.c₁ₙ, Geom.line, color=[colorant"orange"]),
    layer(x=t, y=c₁ₖ.c₁ₚ, Geom.line),
    Guide.title("Variation of solid concentration in electrodes"),
    Guide.xlabel("Time (in s)"),
    Guide.ylabel("c₁ₖ"),
    # Guide.manual_color_key(
    #     "Legend", 
    #     [
    #         "Solid concentration in -ve electode: q₂ᵢₙ", 
    #         "Solid concentration in +ve electode: q₂ᵢₚ"
    #     ], 
    #     ["orange", "blue"]
    # )
)

#### Calculation of Solid Radial Concentration (c₁ₖᵣ)

In [ ]:
c₁ₖᵣ = sm.SolidRadialConcentration(t, tspan, I)

plot(
    layer(x=t, y=c₁ₖᵣ.c₁ₙᵣ, Geom.line, color=[colorant"orange"]),
    layer(x=t, y=c₁ₖᵣ.c₁ₚᵣ, Geom.line),
    Guide.title("Variation of solid radial concentration in electrodes"),
    Guide.xlabel("Time (in s)"),
    Guide.ylabel("c₁ₖᵣ"),
    # Guide.manual_color_key(
    #     "Legend", 
    #     [
    #         "Solid radial concentration in -ve electode: q₂ᵢₙ", 
    #         "solid radial concentration in +ve electode: q₂ᵢₚ"
    #     ], 
    #     ["orange", "blue"]
    # size=Measure[]
    # )
)

#### Calculation of Cell Voltage (V)

In [ ]:
V = sm.V(I, c₁ₖ, c₁ₖᵣ, c₂ᵢₖ, q₂ᵢₖ)

plot(
    layer(x=t, y=V, Geom.line, color=[colorant"orange"]),
    Guide.title("Variation of Cell Voltage"),
    Guide.xlabel("Time (in s)"),
    Guide.ylabel("V"),
)

### Discrete time series current case
In this we will be taking a discrete time series current over a period of 100 seconds with a interval of 10s.

In [ ]:
I_dst = Float64.([100, 100, 0, 0, 0, 0, -100, -100, 0, 0, 0])
t_dst = Float64.(0 : 10: 100)
tspan_dst = 10

plot(
    layer(x=t_dst, y=I_dst, Geom.line, color=[colorant"orange"]),
    Guide.title("Variation of Current"),
    Guide.xlabel("Time (in s)"),
    Guide.ylabel("I"),
)

In [ ]:
q₂ᵢₖ_dst = sm.InterfacialFlux(t_dst, tspan_dst, I_dst)

In [ ]:
c₂ᵢₖ_dst = sm.InterfacialConc(q₂ᵢₖ_dst)

In [ ]:
c₁ₖ_dst = sm.SolidConcentration(t_dst, tspan_dst, I_dst)

In [ ]:
c₁ₖᵣ_dst = sm.SolidRadialConcentration(t_dst, tspan_dst, I_dst)

In [ ]:
V = sm.V(I, c₁ₖ, c₁ₖᵣ, c₂ᵢₖ, q₂ᵢₖ)